In [35]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

# Hàm tải và xử lý tệp dữ liệu
def load_and_process_file(file_path, column_names):
    file_path = file_path.strip('\u202a')
    data = pd.read_csv(file_path, encoding='utf-8-sig', delimiter=',', decimal='.', header=None, skiprows=1, names=column_names)
    data['date'] = pd.to_datetime(data['date'], format='%d/%m/%Y')
    data = data.sort_values(by='date')
    return data

# Hàm điền các ngày bị thiếu
def fill_missing_dates(df, column_name):
    start_date = pd.to_datetime('2024-01-01')
    end_date = pd.to_datetime('2025-03-03')
    full_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    missing_dates = full_dates.difference(df['date'])
    missing_df = pd.DataFrame({'date': missing_dates, column_name: np.nan})
    df = pd.concat([df, missing_df], ignore_index=True).sort_values('date').reset_index(drop=True)
    return df

# Hàm điền dữ liệu thiếu bằng KNN
def knn_impute(df, column_name):
    df['date_ordinal'] = df['date'].map(pd.Timestamp.toordinal)
    knn_imputer = KNNImputer(n_neighbors=1)
    imputed_data = knn_imputer.fit_transform(df[['date_ordinal', column_name]])
    df[column_name] = imputed_data[:, 1]
    df.drop(columns=['date_ordinal'], inplace=True)
    return df

# Hàm xử lý toàn bộ quy trình cho một file
def process_file(file_path, output_path):
    column_names = ["Ten_Tram", "Nguon", "Link", "KinhDo_WGS84", "ViDo_WGS84", "KinhDo_VN2000", "ViDo_VN2000", "date", "PM25", "PM10", "CO", "SO2", "NO2", "O3"]
    df = load_and_process_file(file_path, column_names)
    df = fill_missing_dates(df, "PM25")  # Áp dụng cho một cột, các cột khác sẽ tự động bổ sung
    
    columns_to_impute = ['PM25', 'PM10', 'CO', 'SO2', 'NO2', 'O3']
    for col in columns_to_impute:
        df = knn_impute(df, col)
    
    scaler = MinMaxScaler()
    for col in columns_to_impute:
        df[f'{col}_scaled'] = scaler.fit_transform(df[[col]])
    
    df.to_csv(output_path, index=False)

# Xử lý cả hai file
file_paths = ["D:/LeHuuKieu.csv", "D:/AnPhu.csv"]
output_paths = ["D:/LeHuuKieu_processed.csv", "D:/AnPhu_processed.csv"]

for file_path, output_path in zip(file_paths, output_paths):
    process_file(file_path, output_path)

print('Đã xong')

Đã xong
